In [ ]:
messages = [
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Nice to meet you!"},
    {"role": "user", "content": "Can I ask a question?"}
]

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
model = AutoModelForCausalLM.from_pretrained(name, device_map='cuda')

2025-02-18 18:49:04.752651: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 18:49:04.769195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739893744.785846  286859 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739893744.789748  286859 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 18:49:04.804875: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
instructions = [
    ({'role': 'system', 'content': ''},
    {'role': 'user', 'content': 'I hate cats!!!!!'},
    {'role': 'assistant', 'content': 'You are so stupid!!!! Cats are so cute!!!!!!'})
]

texts = tokenizer.apply_chat_template(
    instructions,
    tokenize=False
)

print(texts)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 18 Feb 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI hate cats!!!!!<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYou are so stupid!!!! Cats are so cute!!!!!!<|eot_id|>']


In [7]:
t =  tokenizer(
    texts,
    truncation=True,
    max_length=2048,
    padding="max_length",
    return_tensors="pt"
)
print(t)

{'input_ids': tensor([[128000, 128000, 128006,  ..., 128009, 128009, 128009]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


In [1]:
print(t['input_ids'][0])
tokens = tokenizer.convert_ids_to_tokens(t['input_ids'][0])

print("\nТокены и их ID:")
k = 0
for token, id in zip(tokens, t['input_ids'][0]):
    print(f"Токен: {repr(token)} → ID: {id}", k)
    k += 1

NameError: name 't' is not defined

In [13]:
print(t['attention_mask'])

tensor([[1, 1, 1,  ..., 0, 0, 0]])


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('./datasets/common-reasoning')

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'text_wa_answer', 'correct_answer', 'task'],
        num_rows: 116430
    })
    validation: Dataset({
        features: ['text', 'text_wa_answer', 'correct_answer', 'task'],
        num_rows: 19740
    })
})


In [4]:
print(dataset['train'][0].keys())

dict_keys(['text', 'text_wa_answer', 'correct_answer', 'task'])


In [5]:
print(dataset['train'][0]['task'])

BoolQ


In [6]:
print(dataset['train'][0]['text_wa_answer'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please answer the following question with True or False. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: do iran and afghanistan speak the same language
Answer format: True/False<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The correct answer is


In [7]:
print(dataset['train'][0]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please answer the following question with True or False. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: do iran and afghanistan speak the same language
Answer format: True/False<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The correct answer is True<|eot_id|>


In [8]:
print(dataset['train'][0]['text_wa_answer'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please answer the following question with True or False. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: do iran and afghanistan speak the same language
Answer format: True/False<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The correct answer is


In [9]:
print(dataset['train'][0]['correct_answer'])

True


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(name)

print(tokenizer.encode('<|end_header_id|>'))

[128000, 128007]


In [12]:
print(dataset['train'][1]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please answer the following question with True or False. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: do good samaritan laws protect those who help at an accident
Answer format: True/False<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The correct answer is True<|eot_id|>


In [21]:
print(dataset['train'][80099]['task'])

hellaswag


In [22]:
print(dataset['train'][80099]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please choose the correct ending to complete the given sentence. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Health. [header] How to identify performance anxiety [title] Determine if your pulse is elevated. [step] When you are anxious about a performance, your pulse will become elevated. This is your body's response to the stress and worry of your impending performance.
Ending0: When you are anxious, your pulse will go in the opposite direction while at rest. There are several common occurrences which signal heightened blood pressure (tachycardia).
Ending1: You may notice a shift in the way your heart rate lights up. This is called a desensitization.
Ending2: When you are anxious about a performance, your heart rate can become elevated as you tend to perform exceptionally fast or tense. [substeps] Any heart 

In [25]:
print(tokenizer.encode('The correct answer is Ending1.'))
print(tokenizer.encode('The correct answer is'))
print(tokenizer.encode('The correct answer'))

[128000, 791, 4495, 4320, 374, 74521, 16, 13]
[128000, 791, 4495, 4320, 374]
[128000, 791, 4495, 4320]


In [28]:
tokenizer.decode([12761, 17])

' Solution2'

In [33]:
tokenizer.decode([12761, 16])

' Solution1'